In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __future__ import division
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, WeekdayLocator,\
    DayLocator, MONDAY,date2num,num2date,AutoDateLocator
from matplotlib.finance import quotes_historical_yahoo_ohlc, candlestick_ohlc,candlestick2_ochl,volume_overlay3

from stockapp import models as stkmd
from dataapp import models as dtamd
from dataapp import tasks as dtatks
from dataapp import libs as dtalibs
from featureapp import libs as ftlibs
from featureapp import models as ftmd
from stockapp import tasks as stktks
from stockapp import libs as stklibs
import featureapp.models as ftmd
import featureapp.tasks as fttks
import queryapp.models as qrymd
import queryapp.tasks as qrytks

import charts.chartservers.libs as chservlibs
import charts.libs as chlibs

from datascience import models as dtscmd
from datascience import tasks as dtsctks
from datascience import libs as dtsclibs


import featureapp as ftapp
import utility as uty
from utility import models as utymd
import itertools as itt
import multiprocessing as mp
from django.db import connection,connections
from django.db import reset_queries
import time 
import pandas as pd
import numpy as np
import inspect
import imp
import datetime
from talib.abstract import *
import utility.models as utmd
import stockapp.libs as stklib
from utility import codemanager as cdmng
from utility import maintenance as mnt
import os 
import json
from django.contrib.auth.models import AnonymousUser
import threading

stk=stkmd.Stockmeta.objects.get(Symbol='TSLA')
Fromdate=pd.datetime(2008,1,1)
Todate=pd.datetime.today()
Trange=pd.date_range(Fromdate,Todate)
Trange=[T.date() for T in Trange if T.weekday()<=4]

import json
# fttks.computefeatuers(stk.id,Trange)

# Charting

In [ ]:
entries=[
    {'Symbol':'TSLA','TF':pd.datetime(2012,1,1).date(),'T0':pd.datetime(2011,1,1).date()  },
    {'Symbol':'AAPL','TF':pd.datetime(2012,1,1).date(),'T0':pd.datetime(2011,1,1).date()  }
]
chservlibs.request_db_charts(entries,5003)
# img=chlibs.CurrentByFutureChart_bydb(entries[0]['T0'],entries[0]['TF'],entries[0]['Symbol'],indicatorlist=(),pricecols=(),querycols=(),featcols=())


# Running Features

In [ ]:
print "---------------- features-------------------"
featurecodes=ftmd.FeatureComputeCode.objects.all()
computecode=featurecodes[0]
computeclass=computecode.importcomputeclass()
CF=computeclass(stk.id,Trange)
CF.computeall(skipdone=True)
# CF.saveall()
print CF.getfeaturelist()

CF.df=CF.addindicators(CF.df,[
        {'name':'SMAstd','timeperiod':20,'colname':'SMAstd20'},
        {'name':'EMAstd','timeperiod':8,'colname':'EMAstd8'},
    ])

# Running Queries

In [ ]:
querycodes=qrymd.QueryComputeCode.objects.all()
computecode=querycodes[0]
computeclass=computecode.importcomputeclass()
CQ=computeclass(stk.id,Trange)
CQ.computeall(skipdone=True)
CQ.saveall()

In [ ]:
CQ.OutcomeCharts()

In [ ]:
CQ.getquerylist()

In [ ]:
CQ.chartfeatures(addpricecols=(),ip=5562,
addfeatcols=[
    ['CCI5','CCI50'],
    ['PastPROFIT10days','PastLOSS10days'],['FutPROFIT10days','FutLOSS10days']
],
addquerycols=[
    'CCICHERRIES',
]
)

# Registering function

In [ ]:
from datascience import libs as dtsclibs
import pandas as pd

@dtsclibs.register_compfunc(RequiredImports=['import pandas as pd','from dataapp import libs as dtalibs'],overwrite_if_exists=False)
def extractdataset(data_id,Symbol):
    """
    @funcName : test
    @input x : an int
    @output df : pd.DataFrame, some random 2 by 2 
    @description : takes an int and then returns a dummy dataframe. This is just for testing purposes
    @Source : 
    def extractdataset(data_id,Symbol):
        window=60
        window_fut=30
        Tfs=map(lambda x: ( (x.date()-pd.Dateoffset(window)).date(),x.date(), (x.date()+pd.Dateoffset(window_fut)).date() ),
                pd.date_range(start=pd.datetime(2010,1,1),end=pd.datetime.today(),freq='W-MON') )

        N=len(Tfs)
        dfinstants=pd.DataFrame({'T0':map(lambda x: x[0],Tfs),'TF':map(lambda x: x[1],Tfs),'Symbol':[Symbol]*N})
        X,X_Meta=dtalibs.Getbatchdata(dfinstants)

        dfinstants=pd.DataFrame({'T0':map(lambda x: x[1],Tfs),'TF':map(lambda x: x[2],Tfs),'Symbol':[Symbol]*N})
        Y,Y_Meta=dtalibs.Getbatchdata(dfinstants)

        shard=dtscmd.DataShard(Data__id=data_id)
        shard.Info['X_Meta']=X_Meta
        shard.Info['Y_Meta']=Y_Meta
        shard.save()

        np.savez_compressed(shard.shardpath(),X=X,Y=Y)
    """
    
    window=60
    window_fut=30
    Tfs=map(lambda x: ( (x.date()-pd.Dateoffset(window)).date(),x.date(), (x.date()+pd.Dateoffset(window_fut)).date() ),
            pd.date_range(start=pd.datetime(2010,1,1),end=pd.datetime.today(),freq='W-MON') )
    
    N=len(Tfs)
    dfinstants=pd.DataFrame({'T0':map(lambda x: x[0],Tfs),'TF':map(lambda x: x[1],Tfs),'Symbol':[Symbol]*N})
    X,X_Meta=dtalibs.Getbatchdata(dfinstants)
    
    dfinstants=pd.DataFrame({'T0':map(lambda x: x[1],Tfs),'TF':map(lambda x: x[2],Tfs),'Symbol':[Symbol]*N})
    Y,Y_Meta=dtalibs.Getbatchdata(dfinstants)
    
    shard=dtscmd.DataShard(Data__id=data_id)
    shard.Info['X_Meta']=X_Meta
    shard.Info['Y_Meta']=Y_Meta
    shard.save()
    
    np.savez_compressed(shard.shardpath(),X=X,Y=Y)

extractdataset.id


# Creating initial Stock price dataset

In [1]:
from datascience import models as dtscmd
from datascience import tasks as dtsctks
from datascience import libs as dtsclibs

projid,dataid=dtsclibs.register_dataset(project_Name="PredictReturn",project_Info={'description': "Data taken on every Monday. 360 days back and 60 days forward"},
                                        Datatype='RawProcessed',GroupName="AllStocks",tag="1",
                                        data_format='npz',Modeltype='Regression',
                                        TransformedFromDataId=None,TransFuncId=None, use_project_ifexists=True)

dtsctks.CreateStockData_2(360,60,dataid,None)

Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Project  PredictReturn  already exists
The dataset already exists
('project id', 'data id')  :  (7, 6)
Working on Symbol  KEYW
Working on Symbol  SIG
Working on Symbol  NLSN
Working on Symbol  AGIO
Working on Symbol  AZPN
Working on Symbol  DKS
Working on Symbol  ARMK
Working on Symbol  SEAS
Working on Symbol  CWEI
Working on Symbol  UMH
Working on Symbol  AL
Working on Symbol  DCO
Working on Symbol  PICO
Working on Symbol  TER
Working on Symbol  AFG
Working on Symbol  DAR
Working on Symbol  POL
Working on Symbol  DOOR
Working on Symbol  TSEM
Working on Symbol  AVT
Working on Symbol  AAT
Working on Symbol  MINI
Working on Symbol  DLTR
Working on Symbol  AZO
Working on Symbol  ST
Working on Symbol  HMN
Working on Symbol  UBNT
Working on Symbol  RESI
Working on Symbol  MTB
Working on Symbol  PRN
Working on Symbol  SONC
Working on Symbol  AVTC
Working on Symbol  DTLK
Working on Symbol  DPM
Working on Symbol  CALL
Working on Symbol  CMI
Working on Symbol  AMPE
Working on Symbol  ACFN
Worki

Working on Symbol  BABY
Working on Symbol  ETG
Working on Symbol  JBLU
Working on Symbol  MMM
Working on Symbol  BSMX
Working on Symbol  SPRT
Working on Symbol  COVS
Working on Symbol  NTAP
Working on Symbol  ASDR
Working on Symbol  NTIP
Working on Symbol  TXI
Working on Symbol  GTI
Working on Symbol  TCX
Working on Symbol  CRAI
Working on Symbol  MYJ
Working on Symbol  TENX
Working on Symbol  PFNX
Working on Symbol  OCUL
Working on Symbol  NPV
Working on Symbol  SPKE
Working on Symbol  ICD
Working on Symbol  GLOB
Working on Symbol  CTLT
Working on Symbol  HDG
Working on Symbol  MIK
Working on Symbol  WLKP
Working on Symbol  ERI
Working on Symbol  AVAL
Working on Symbol  ENI
Working on Symbol  BSTC
Working on Symbol  BCBF
Working on Symbol  BSI
Working on Symbol  BUI
Working on Symbol  CUDA
Working on Symbol  MLNX
Working on Symbol  LOJN
Working on Symbol  MOV
Working on Symbol  TEAR
Working on Symbol  SON
Working on Symbol  TG
Working on Symbol  TST
Working on Symbol  JOE
Working on S

Working on Symbol  FPRX
Working on Symbol  VRTS
Working on Symbol  NYCB
Working on Symbol  CTL
Working on Symbol  NWFL
Working on Symbol  MSCC
Working on Symbol  PIKE
Working on Symbol  BZC
Working on Symbol  NVRO
Working on Symbol  MPG
Working on Symbol  NVSL
Working on Symbol  NAII
Working on Symbol  OUBS
Working on Symbol  MONY
Working on Symbol  NBN
Working on Symbol  MMD
Working on Symbol  OSN
Working on Symbol  MLVF
Working on Symbol  NCQ
Working on Symbol  NNP
Working on Symbol  NQC
Working on Symbol  NGE
Working on Symbol  TGTX
Working on Symbol  CNSL
Working on Symbol  NEPT
Working on Symbol  WGBS
Working on Symbol  REDF
Working on Symbol  MCOX
Working on Symbol  PBFX
Working on Symbol  USAC
Working on Symbol  RRMS
Working on Symbol  CELP
Working on Symbol  EMHY
Working on Symbol  UAN
Working on Symbol  CPHR
Working on Symbol  ALTR
Working on Symbol  WNRL
Working on Symbol  BWFG
Working on Symbol  BWZ
Working on Symbol  BRY
Working on Symbol  BSJJ
Working on Symbol  BVX
Workin

Working on Symbol  PTH
Working on Symbol  PAI
Working on Symbol  JQC
Working on Symbol  EIRL
Working on Symbol  LCI
Working on Symbol  DFS
Working on Symbol  TI
Working on Symbol  CBG
Working on Symbol  CBT
Working on Symbol  BBCN
Working on Symbol  NCTY
Working on Symbol  GPM
Working on Symbol  ACRX
Working on Symbol  NRZ
Working on Symbol  BGSF
Working on Symbol  ISSC
Working on Symbol  TREC
Working on Symbol  XIV
Working on Symbol  DAKP
Working on Symbol  CCCL
Working on Symbol  MNGA
Working on Symbol  CEMI
Working on Symbol  SJW
Working on Symbol  CAPL
Working on Symbol  AAU
Working on Symbol  CEMP
Working on Symbol  CAH
Working on Symbol  NBCB
Working on Symbol  NWHM
Working on Symbol  CMD
Working on Symbol  BTX
Working on Symbol  COST
Working on Symbol  POZN
Working on Symbol  NIQ
Working on Symbol  SBS
Working on Symbol  NOM
Working on Symbol  CNW
Working on Symbol  QTM
Working on Symbol  PKBK
Working on Symbol  SCX
Working on Symbol  ITRN
Working on Symbol  KEYS
Working on Symb

...

Working on Symbol  ATRC
Working on Symbol  RWWI
Working on Symbol  SGMZ
Working on Symbol  SCM
Working on Symbol  RYFL
Working on Symbol  RZG
Working on Symbol  SAGG
Working on Symbol  SBAZ
Working on Symbol  SVBL
Working on Symbol  RYH
Working on Symbol  SALM
Working on Symbol  SAR
Working on Symbol  SAMG
Working on Symbol  RWK
Working on Symbol  SCOO
Working on Symbol  SCAI
Working on Symbol  SOBS
Working on Symbol  SAA
Working on Symbol  ORGN
Working on Symbol  NWBN
Working on Symbol  SCKT
Working on Symbol  RZV
Working on Symbol  SCSG
Working on Symbol  SCJ
Working on Symbol  SCND
Working on Symbol  SEB
Working on Symbol  SEBC
Working on Symbol  SGA
Working on Symbol  SBFG
Working on Symbol  SEV
Working on Symbol  SFGP
Working on Symbol  SBSA
Working on Symbol  SBHO
Working on Symbol  SCAF
Working on Symbol  SCVE
Working on Symbol  SFN
Working on Symbol  SGGH
Working on Symbol  SCRI
Working on Symbol  SBFC
Working on Symbol  SLYG
Working on Symbol  SLVO
Working on Symbol  SLY
Worki

Working on Symbol  KYN
Working on Symbol  ACMP
Working on Symbol  STWD
Working on Symbol  ABB
Working on Symbol  DWRE
Working on Symbol  WSB
Working on Symbol  ZPPB
Working on Symbol  MKSI
Working on Symbol  RLGT
Working on Symbol  SBUX
Working on Symbol  POWR
Working on Symbol  NPO
Working on Symbol  NEWT
Working on Symbol  TSRA
Working on Symbol  YDIV
Working on Symbol  XSW
Working on Symbol  XTL
Working on Symbol  WPS
Working on Symbol  YCL
Working on Symbol  WMSI
Working on Symbol  XTN
Working on Symbol  WIRX
Working on Symbol  TNXP
Working on Symbol  FDUS
Working on Symbol  CUI
Working on Symbol  MHO
Working on Symbol  MLNK
Working on Symbol  SANW
Working on Symbol  MYL
Working on Symbol  NGL
Working on Symbol  SPNS
Working on Symbol  SCLN
Working on Symbol  SPB
Working on Symbol  OZM
Working on Symbol  GASS
Working on Symbol  HZO
Working on Symbol  SPCO
Working on Symbol  CKSW
Working on Symbol  COV
Working on Symbol  TLI
Working on Symbol  WMK
Working on Symbol  INAP
Working on 

Working on Symbol  ISF
Working on Symbol  IDI
Working on Symbol  TPI
Working on Symbol  MMP
Working on Symbol  ANAC
Working on Symbol  FISI
Working on Symbol  HOS
Working on Symbol  NFJ
Working on Symbol  AGX
Working on Symbol  CGG
Working on Symbol  XRX
Working on Symbol  INFU
Working on Symbol  AEZS
Working on Symbol  HASI
Working on Symbol  DHR
Working on Symbol  ALNY
Working on Symbol  XNY
Working on Symbol  WASH
Working on Symbol  MPX
Working on Symbol  OHAI
Working on Symbol  LIOX
Working on Symbol  ERB
Working on Symbol  SXL
Working on Symbol  PBI
Working on Symbol  CHFN
Working on Symbol  APT
Working on Symbol  HMY
Working on Symbol  SREV
Working on Symbol  SCHX
Working on Symbol  EEV
Working on Symbol  EXPO
Working on Symbol  GPN
Working on Symbol  MDLY
Working on Symbol  MSA
Working on Symbol  ZLTQ
Working on Symbol  FSM
Working on Symbol  OFG
Working on Symbol  DDC
Working on Symbol  WPC
Working on Symbol  USEG
Working on Symbol  DIOD
Working on Symbol  OPK
Working on Symbol

Working on Symbol  BLMT
Working on Symbol  CPST
Working on Symbol  HE
Working on Symbol  BOOT
Working on Symbol  TZOO
Working on Symbol  AXAS
Working on Symbol  CLNY
Working on Symbol  CNQ
Working on Symbol  CRIS
Working on Symbol  EPD
Working on Symbol  ESTE
Working on Symbol  SIBC
Working on Symbol  EVJ
Working on Symbol  SCHV
Working on Symbol  ENRJ
Working on Symbol  EVP
Working on Symbol  EVY
Working on Symbol  EVO
Working on Symbol  ERS
Working on Symbol  EVX
Working on Symbol  HTWR
Working on Symbol  APPY
Working on Symbol  CSL
Working on Symbol  DL
Working on Symbol  EMCB
Working on Symbol  UFI
Working on Symbol  ENZL
Working on Symbol  EML
Working on Symbol  EUFX
Working on Symbol  ENGA
Working on Symbol  EMZ
Working on Symbol  ERJ
Working on Symbol  SMCI
Working on Symbol  TREX
Working on Symbol  BID
Working on Symbol  BLX
Working on Symbol  HAS
Working on Symbol  CLNE
Working on Symbol  AWR
Working on Symbol  HMST
Working on Symbol  BTI
Working on Symbol  FHCO
Working on Sym

Working on Symbol  SCI
Working on Symbol  CRK
Working on Symbol  CSIQ
Working on Symbol  SFE
Working on Symbol  DOV
Working on Symbol  GNC
Working on Symbol  SNDK
Working on Symbol  GTAT
Working on Symbol  ARE
Working on Symbol  CVT
Working on Symbol  ATI
Working on Symbol  BEN
Working on Symbol  HURN
Working on Symbol  ZQK
Working on Symbol  SJI
Working on Symbol  UFPI
Working on Symbol  WAL
Working on Symbol  NCI
Working on Symbol  FHRT
Working on Symbol  NDSN
Working on Symbol  KNL
Working on Symbol  DBO
Working on Symbol  YHOO
Working on Symbol  RSG
Working on Symbol  WETF
Working on Symbol  DSW
Working on Symbol  ZEP
Working on Symbol  CUR
Working on Symbol  TAL
Working on Symbol  S
Working on Symbol  YGE
Working on Symbol  GPDB
Working on Symbol  MSO
Working on Symbol  AIZ
Working on Symbol  EIM
Working on Symbol  TWTC
Working on Symbol  REV
Working on Symbol  TPH
Working on Symbol  VKI
Working on Symbol  IYZ
Working on Symbol  CRR
Working on Symbol  BRE
Working on Symbol  BLT
Wo

In [2]:
print dtscmd.Data.objects.filter(Project__id=7,GroupName="AllStocks",tag="1",Datatype='RawProcessed',
                           Dataformat='npz',Modeltype='Regression')
data = dtscmd.Data.objects.get(Project__id=7,GroupName="AllStocks",tag="1",Datatype='RawProcessed',
                           Dataformat='npz',Modeltype='Regression')

data.datapath()

<QuerySet [<Data: PredictReturn AllStocks 1 Regression RawProcessed npz>]>


u'/home/venkat/n.adurthi@gmail.com/repos/trade_analytics/trade_analytics/bigdata/datascience/Projects/PredictReturn/Data/RawProcessed/AllStocks_1'

In [3]:
dtscmd.DataShard.objects.filter(Data=data).delete()

coercing to Unicode: need string or buffer, tuple found
coercing to Unicode: need string or buffer, tuple found
coercing to Unicode: need string or buffer, tuple found
coercing to Unicode: need string or buffer, tuple found
coercing to Unicode: need string or buffer, tuple found
coercing to Unicode: need string or buffer, tuple found
coercing to Unicode: need string or buffer, tuple found
coercing to Unicode: need string or buffer, tuple found
coercing to Unicode: need string or buffer, tuple found
coercing to Unicode: need string or buffer, tuple found
coercing to Unicode: need string or buffer, tuple found
coercing to Unicode: need string or buffer, tuple found
coercing to Unicode: need string or buffer, tuple found
coercing to Unicode: need string or buffer, tuple found
coercing to Unicode: need string or buffer, tuple found
coercing to Unicode: need string or buffer, tuple found
coercing to Unicode: need string or buffer, tuple found
coercing to Unicode: need string or buffer, tupl

(32, {u'datascience.DataShard': 32})

In [14]:
dtscmd.DataShard.objects.all()

<QuerySet []>

# Functions on shards

In [1]:
from datascience import models as dtscmd
from datascience import tasks as dtsctks
from datascience import libs as dtsclibs
from celery import group
import time
import pickle as pkl

@dtsclibs.register_compfunc(Group='test',overwrite_if_exists=True)
def GetMeta(shardId):
    print "----------------------"
    print shardId
    print "----------------------"
    
    shard=dtscmd.DataShard.objects.get(id=shardId)
    X,Y,Meta=shard.getdata()
    return Meta

M=dtsctks.applyfunc2data(GetMeta.id,6,wait=True)


Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


over writing previous function
saving function :  GetMeta
function id =  19
302 0 304
301 0 304
300 0 304
299 0 304
298 0 304
297 0 304
296 0 304
295 0 304
294 0 304
293 0 304
292 0 304
291 0 304
290 0 304
289 0 304
288 0 304
287 0 304
286 0 304
285 0 304
284 0 304
283 0 304
282 0 304
281 0 304
280 0 304
279 0 304
278 0 304
277 0 304
276 0 304
275 0 304
274 0 304
273 0 304
272 0 304
271 0 304
270 0 304
269 0 304
268 0 304
267 0 304
266 0 304
265 0 304
264 0 304
263 0 304
262 0 304
261 0 304
260 0 304
259 0 304
258 0 304
257 0 304
256 0 304
255 0 304
254 0 304
253 0 304
252 0 304
251 0 304
250 0 304
249 0 304
248 0 304
247 0 304
246 0 304
245 0 304
244 0 304
243 0 304
242 0 304
241 0 304
240 0 304
239 0 304
238 0 304
237 0 304
236 0 304
235 0 304
234 0 304
233 0 304
232 0 304
231 0 304
230 0 304
229 0 304
228 0 304
227 0 304
226 0 304
225 0 304
224 0 304
223 0 304
222 0 304
221 0 304
220 0 304
219 0 304
218 0 304
217 0 304
216 0 304
215 0 304
214 0 304
213 0 304
212 0 304
211 0 304
210 

302 294 304
301 294 304
300 294 304
299 295 304
298 296 304
297 297 304
296 298 304
295 299 304
294 300 304
293 301 304
292 301 304
291 301 304
290 301 304
289 301 304
288 301 304
287 301 304
286 301 304
285 301 304
284 301 304
283 301 304
282 301 304
281 301 304
280 301 304
279 301 304
278 301 304
277 301 304
276 301 304
275 301 304
274 301 304
273 301 304
272 301 304
271 301 304
270 301 304
269 301 304
268 301 304
267 301 304
266 301 304
265 301 304
264 301 304
263 301 304
262 301 304
261 301 304
260 301 304
259 301 304
258 301 304
257 301 304
256 301 304
255 301 304
254 301 304
253 301 304
252 301 304
251 301 304
250 301 304
249 301 304
248 301 304
247 301 304
246 301 304
245 301 304
244 301 304
243 301 304
242 301 304
241 301 304
240 301 304
239 301 304
238 301 304
237 301 304
236 301 304
235 301 304
234 301 304
233 301 304
232 301 304
231 301 304
230 301 304
229 301 304
228 301 304
227 301 304
226 301 304
225 301 304
224 301 304
223 301 304
222 301 304
221 301 304
220 301 304
219 

In [ ]:
dtscmd.DataShard.objects.filter(Data__id=6,id__lte=6036).delete()

# Creating derived DataSet

In [10]:
from datascience import models as dtscmd
dtscmd.Project.objects.all()
# data=dtscmd.Data.objects.filter(Project=pr)[0]

<QuerySet [<Project: PredictReturn>]>

In [9]:
data.delete()

Signal recieived to delete all data for :  PredictReturn OnlyTSLA 1 Regression RawProcessed npz


(1,
 {u'datascience.Data': 1,
  u'datascience.DataShard': 0,
  u'datascience.ModelMetrics': 0})